In [1]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


In [ ]:
Instance

In [226]:
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
#X = preprocess.preprocess(X)

277792


In [ ]:
Profile

In [232]:
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
X, y = dataset.get_data('age')
#X, y = dataset.get_data('gender')
print len(X)
#print X[0]
#X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


In [7]:
from nltk.internals import find_binary
_megam_bin = None
def config_megam(bin=None):
    """
    Configure NLTK's interface to the ``megam`` maxent optimization
    package.
 
    :param bin: The full path to the ``megam`` binary.  If not specified,
        then nltk will search the system for a ``megam`` binary; and if
        one is not found, it will raise a ``LookupError`` exception.
    :type bin: str
    """
    global _megam_bin
    _megam_bin = find_binary(
        'megam', bin,
        env_vars=['MEGAM'],
        binary_names=['megam.opt', 'megam', 'megam_686', 'megam_i686.opt'],
        url='http://www.umiacs.umd.edu/~hal/megam/index.html')
from nltk import MaxentClassifier


In [9]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from pan.features import SOA_Model2
soac = features.SOAC_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None)
svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced', probability=False)
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies), 
#                          ('soa', soa), ('soac', soac)])+
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies)])
pipe1 = Pipeline([('soac',soac), ('svm', svm)])
pipe1.steps

[('soac', SOAC_Model2(max_df=1.0, max_features=None, min_df=1, thres=0.1,
        tokenizer_var='sklearn')),
 ('svm', SVC(C=0.1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_recall_fscore_support, classification_report

In [250]:
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
num_folds = 4
split = 0.25
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y)
for i, x in enumerate(X_train):
    if len(x)==0:
        X_train.remove(x)
        y_train.remove(y_train[i])
for i, x in enumerate(X_cv):
    if len(x)==0:
        X_cv.remove(x)
        y_cv.remove(y_cv[i])
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)
X_meta, X_cv, y_meta, y_cv = train_test_split(X_cv, y_cv, test_size=0.5, stratify=y_cv)
print len(X_train), len(X_cv), len(X_meta), len(X_cv) + len(X_train) + len(X_meta), len(X)
params = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100]}
params = {}
grid_cv = GridSearchCV(pipe1, param_grid=params, verbose=1, n_jobs=-1, cv=num_folds, refit=True)
grid_cv.fit(X_train, y_train)
#grid_cv.fit(X_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)
pipe1 = grid_cv.best_estimator_

326 110 436 436
326 55 55 436 436


In [53]:
from nltk.corpus import names
from nltk import MaxentClassifier
from nltk import classify
import random

names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)
 
def gender_features3(name):
        features = {}
        features["fl"] = name[0].lower()
        features["ll"] = name[-1].lower()
        features["fw"] = name[:2].lower()
        features["lw"] = name[-2:].lower()
        return features

 
featuresets = [(gender_features3(n), g) for (n, g) in names]
 
train_set, test_set = featuresets[500:], featuresets[:500]
 
me3_megam_classifier = MaxentClassifier.train(train_set, "megam")
classify.accuracy(me3_megam_classifier, test_set)

0.826

In [205]:
import regex as re
import nltk
import numpy
from textblob.tokenizers import WordTokenizer
from sklearn.base import BaseEstimator, TransformerMixin
from pan.misc import _twokenize


def tokenization2(text):

        import re

        emoticons_str = r"""
        (?:
          [:=;] # Eyes
          [oO\-]? # Nose (optional)
          [D\)\]\(\]/\\OpP] # Mouth
        )"""

        regex_str = [
            emoticons_str,
            #r'<[^>]+>',  # HTML tags
            #r'(?:@[\w_]+)',  # @-mentions
            #r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
            #r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs
            #r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
            #r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
            #r'(?:[\w_]+)',  # other words
            #r'(?:\S)'  # anything else
        ]
        tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
        emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

        return [token if emoticon_re.search(token) else token.lower() for token in tokens_re.findall(text)]

class CountHash(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of twitter-style hashes. """

    pat = re.compile(r'(?<=\s+|^)#\w+', re.UNICODE)

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count hashes in
        :returns: list of counts for each text

        """
        return [[len(CountHash.pat.findall(text)) / float(len(text))] for text in texts]


class CountReplies(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of twitter-style @replies. """

    pat = re.compile(r'(?<=\s+|^)@\w+', re.UNICODE)

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count replies in
        :returns: list of counts for each text

        """
        return [[len(CountReplies.pat.findall(text)) / float(len(text))] for text in texts]


class CountURLs(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of URL links from text. """

    pat = re.compile(r'((https?|ftp)://[^\s/$.?#].[^\s]*)')

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count URLs in
        :returns: list of counts for each text

        """
        return [[len(CountURLs.pat.findall(text)) / float(len(text))] for text in texts]


class CountCaps(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of capital letters from text. """

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital letters in
        :returns: list of counts for each text

        """
        return [[sum(c.isupper() for c in text)] for text in texts]


class CountWordCaps(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of capital words from text. """

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: list of counts for each text

        """
        return [[sum(w.isupper() for w in nltk.word_tokenize(text))]
                for text in texts]


class CountWordLength(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of word length from text. """

    def __init__(self, span):
        """ Initialize this feature extractor
        :span: tuple - range of lengths to count

        """
        self.span = span

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count word lengths in
        :returns: list of counts for each text

        """

        mini = self.span[0]
        maxi = self.span[1]
        num_counts = maxi - mini
        # wt = WordTokenizer()
        tokens = [tokenization(text) for text in texts]
        text_len_dist = []
        for line_tokens in tokens:
            counter = [0] * num_counts
            for word in line_tokens:
                word_len = len(word)
                if mini <= word_len <= maxi:
                    counter[word_len - 1] += 1
            text_len_dist.append([each for each in counter])
        return text_len_dist


class CountTokens(BaseEstimator, TransformerMixin):

    """ Model that extracts a counter of capital words from text. """

    def __init__(self):
        self.l = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                  'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3',
                  '4', '5', '6', '7', '8', '9', '!', '.', ':', '?',#]
                  ';', ',', ')', '(', '-', '%', '$', '#', '@', '^',
                  '&', '*', '=', '+', '/', '"', "'", '<', '>', '|',
                  '~', '`']

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: list of counts for each text

        """
        l = self.l
        return [[text.lower().count(token)/float(len(text)) for token in l]
                for text in texts]

In [240]:
import re

POSITIVE = ["*O", "*-*", "*O*", "*o*", "* *",
            ":P", ":D", ":d", ":p",
            ";P", ";D", ";d", ";p",
            ":-)", ";-)", ":=)", ";=)",
            ":<)", ":>)", ";>)", ";=)",
            "=}", ":)", "(:;)",
            "(;", ":}", "{:", ";}",
            "{;:]",
            "[;", ":')", ";')", ":-3",
            "{;", ":]",
            ";-3", ":-x", ";-x", ":-X",
            ";-X", ":-}", ";-=}", ":-]",
            ";-]", ":-.)",
            "^_^", "^-^"]

NEGATIVE = [":(", ";(", ":'(",
            "=(", "={", "):", ");",
            ")':", ")';", ")=", "}=",
            ";-{{", ";-{", ":-{{", ":-{",
            ":-(", ";-(",
            ":,)", ":'{",
            "[:", ";]"
            ]

class EmoticonsCounter(object):

    
    """ Model that extracts a counter emoticons """

    def __init__(self, POSITIVE, NEGATIVE):
        self.l = POSITIVE + NEGATIVE

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: list of counts for each text

        """
        l = self.l
        #print texts
        return [[text.count(emo) for emo in l]
                for text in texts]

In [181]:
pt = ParseTweet(X[3])
pt.getAttributeEmoticon(X[3])

[]

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(min_df=5, max_df=0.9, max_features=100, tokenizer=_twokenize.tokenizeRawTweetText)

In [288]:
a = CountTokens()
a = EmoticonsCounter(POSITIVE, NEGATIVE)
dd = numpy.array(a.transform(X))
dd.shape

(436, 68)

In [289]:
import pandas as pd
data = pd.DataFrame(dd, columns=a.l)
data["class"] = y
print(data.describe())

               *O  *-*  *O*  *o*         * *          :P          :D  \
count  436.000000  436  436  436  436.000000  436.000000  436.000000   
mean     0.011468    0    0    0    0.027523    0.809633    2.256881   
std      0.126333    0    0    0    0.324018    4.312292   11.214649   
min      0.000000    0    0    0    0.000000    0.000000    0.000000   
25%      0.000000    0    0    0    0.000000    0.000000    0.000000   
50%      0.000000    0    0    0    0.000000    0.000000    0.000000   
75%      0.000000    0    0    0    0.000000    0.000000    0.000000   
max      2.000000    0    0    0    6.000000   51.000000  144.000000   

               :d          :p          ;P     ...      ;-{{  ;-{  :-{{  \
count  436.000000  436.000000  436.000000     ...       436  436   436   
mean     0.050459    0.291284    1.529817     ...         0    0     0   
std      0.467052    1.772074    3.595851     ...         0    0     0   
min      0.000000    0.000000    0.000000     ...      

In [300]:
grouped = data.groupby('class')
means = grouped.mean().T

import matplotlib.pyplot as plt
### BAR PLOTS OF MEAN VALUE OF FEATURES FOR EACH CLASS ######

grouped = data.groupby('class')
plt.figure()
grouped.mean().T.plot(kind='barh', figsize=(60,10))
plt.savefig('test1.png')
plt.show()

In [242]:
def feat_to_fi(feat_vector, names):
    features = {}
    #print feat_vector
    #print len(feat_vector)
    for i in xrange(len(feat_vector)):
        features[names[i]] = feat_vector[i]
    return features
    

In [270]:
#trained_soac = pipe1.steps[0][1]
#Transf = trained_soac.transform(X_train + X_cv)
Transf = a.transform(X_train+X_meta +X_cv)
#Transf = tf.fit_transform(X_train + X_cv).todense()
#from sklearn.preprocessing import normalize
#Transf = normalize(Transf, axis=1, norm='l1')
Transf = numpy.array(Transf)
print Transf[0,:]
y_all = y_train + y_meta + y_cv
featuresets = ([(feat_to_fi(Transf[i,:], a.l), y_all[i]) for i in xrange(Transf.shape[0]) ])
train_set, test_set = featuresets[:len(X_train)], featuresets[len(X_train):]
meta_set, test_set = test_set[:len(X_meta)], test_set[len(X_meta):]
#mec = MaxentClassifier.train(train_set, "megam")
#mec = nltk.classify.MaxentClassifier.train(train_set, 'GIS', trace=0, max_iter=1000)

[ 0  0  0  0  0  4  4  0  8  1  0  6  6  0  0  0  0  0  0  0  0  0 14  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
tf.get_feature_names()

In [301]:
scores = []
gauss = []
for i in xrange(1000):
    gauss.append(random.random()*random.randint(0,10))
    mec = MaxentClassifier.train(train_set, "megam", gaussian_prior_sigma=gauss[-1])
    scores.append(classify.accuracy(mec, meta_set))
    print "Best prior: %0.3f with score: %0.2f " % (gauss[scores.index(max(scores))],max(scores)) 
#mec = nltk.classify.MaxentClassifier.train(train_set, 'GIS', trace=0, max_iter=1000)
#classify.accuracy(mec, test_set)

Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 3.525 with score: 0.51 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score: 0.53 
Best prior: 0.000 with score

In [302]:
mec = MaxentClassifier.train(train_set, "megam", gaussian_prior_sigma=gauss[scores.index(max(scores))])

In [303]:
mec.show_most_informative_features(n=100)

  -1.583 ;(==0 and label is '18-24'
   1.167 ;-)==0 and label is '18-24'
  -0.980 ;D==0 and label is '50-64'
   0.893 :(==0 and label is '50-64'
   0.872 ;P==1 and label is '18-24'
   0.854 :D==4 and label is '18-24'
  -0.833 ;D==0 and label is '18-24'
   0.831 :)==1 and label is '18-24'
  -0.813 (;==0 and label is '18-24'
   0.813 :P==0 and label is '50-64'
  -0.804 :P==0 and label is '25-34'
   0.801 ):==0 and label is '18-24'
   0.799 *O==0 and label is '35-49'
   0.780 ;d==6 and label is '35-49'
  -0.769 :d==0 and label is '50-64'
  -0.716 ;p==0 and label is '25-34'
   0.686 ;p==0 and label is '50-64'
   0.684 ;(==0 and label is '35-49'
   0.680 ;}==0 and label is '35-49'
   0.650 :)==2 and label is '50-64'
   0.649 );==0 and label is '25-34'
  -0.621 ;d==0 and label is '25-34'
  -0.618 :)==1 and label is '35-49'
   0.599 ;p==6 and label is '25-34'
   0.597 :)==6 and label is '25-34'
  -0.586 ;d==2 and label is '35-49'
   0.580 ;(==2 and label is '25-34'
   0.577 :D==4 and label is

In [304]:
mec.explain(train_set[0][0])

  Feature                                            25-34   35-49   18-24   50-64
  --------------------------------------------------------------------------------
  ;p==6 (1)                                          0.599
  :D==4 (1)                                         -0.519
  :)==14 (1)                                         0.483
  ;D==0 (1)                                          0.395
  ;d==6 (1)                                          0.347
  :d==0 (1)                                          0.343
  :-)==0 (1)                                        -0.308
  )===0 (1)                                          0.282
  :(==1 (1)                                          0.270
  =(==0 (1)                                         -0.244
  (;==0 (1)                                          0.229
  ;(==0 (1)                                          0.222
  :]==0 (1)                                         -0.194
  ):==0 (1)                                          0.183
  ;-(==0

In [305]:
mec.classify_many([test_sample[0] for test_sample in test_set])

['35-49',
 '50-64',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '25-34',
 '35-49',
 '35-49',
 '35-49',
 '18-24',
 '35-49',
 '35-49',
 '35-49',
 '25-34',
 '25-34',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '25-34',
 '35-49',
 '35-49',
 '25-34',
 '50-64',
 '50-64',
 '35-49',
 '35-49',
 '50-64',
 '35-49',
 '35-49',
 '25-34',
 '35-49',
 '35-49',
 '35-49',
 '25-34',
 '35-49',
 '35-49',
 '18-24',
 '50-64',
 '50-64',
 '25-34',
 '25-34',
 '25-34',
 '35-49',
 '35-49',
 '35-49',
 '35-49',
 '50-64',
 '35-49',
 '35-49',
 '35-49']

In [276]:
len(y_cv), len(meta_set)

(55, 55)

In [306]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
#predict = model.predict(X_cv)
predict = mec.classify_many([test_sample[0] for test_sample in test_set])
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.381818181818
Confusion matrix :
 [[ 0  1  3  0  0]
 [ 2  5  9  1  0]
 [ 0  3 15  5  0]
 [ 0  1  8  1  0]
 [ 0  0  1  0  0]]
Classification report :
              precision    recall  f1-score   support

      18-24       0.00      0.00      0.00         4
      25-34       0.50      0.29      0.37        17
      35-49       0.42      0.65      0.51        23
      50-64       0.14      0.10      0.12        10
      65-xx       0.00      0.00      0.00         1

avg / total       0.35      0.38      0.35        55

